In [11]:
# Pre-Processing Data and Converting into a Recommendation Model

In [3]:
#importing libraries
import numpy as np
import pandas as pd

In [4]:
#Reading the csv file
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
credits.rename(columns = {'movie_id':'id'}, inplace = True)

In [8]:
movies.rename(columns = {'title_x':'title'}, inplace = True)

In [9]:
movies = movies.merge(credits,on='id')

In [10]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
##Overview will help us know the content, similarity for a movie

In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [13]:
#Columns that will help us build the recommendation model
movies = movies[["genres", "id", "title_x", "overview", "keywords", "cast", "crew"]]

In [14]:
movies.head(1)

,genres,id,title_x,overview,keywords,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [15]:
movies.rename(columns = {'id':'movie_id'}, inplace = True)

In [16]:
movies.rename(columns = {'title_x':'title'}, inplace = True)

In [17]:
movies.head(1)

,genres,movie_id,title,overview,keywords,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:
#checking if any null value is present
movies.isnull().sum()

genres      0
movie_id    0
title       0
overview    3
keywords    0
cast        0
crew        0
dtype: int64

In [19]:
movies.dropna(inplace=True)

In [20]:
movies.isnull().sum()

genres      0
movie_id    0
title       0
overview    0
keywords    0
cast        0
crew        0
dtype: int64

In [21]:
movies.duplicated().sum()

0

In [22]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [23]:
#function to get list of name from dictionary
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [24]:
movies['genres'] = movies['genres'].apply(convert)

In [25]:
movies.head(1)

,genres,movie_id,title,overview,keywords,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [26]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(1)

,genres,movie_id,title,overview,keywords,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [27]:
#This helper function will help us in getting the first 3 cast name of every movie.
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [28]:
movies['cast'] = movies['cast'].apply(convert3)

In [29]:
movies.head(1)

,genres,movie_id,title,overview,keywords,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [30]:
#This helper function will help us in finding the director of each movie from crew column.
def find_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [31]:
movies['crew'] = movies['crew'].apply(find_director)

In [32]:
movies.head()

,genres,movie_id,title,overview,keywords,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,Spectre,A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,John Carter,"John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [33]:
#converting string into a list using lambda function
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [34]:
movies.head(2)

,genres,movie_id,title,overview,keywords,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [35]:
#Transformation of 2 strings into 1(ex: Johnny Deep - JohnnyDeep)
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [36]:
#creating TAGs which is created by concatenating all the important column
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [37]:
movies.head(2)

,genres,movie_id,title,overview,keywords,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,"[Adventure, Fantasy, Action]",285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, ocean, drugabuse,..."


In [38]:
df = movies[['movie_id', 'title', 'tags']]
df

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."
...,...,...,...
4798,9367,El Mariachi,"[Action, Crime, Thriller, unitedstates–mexicob..."
4799,72766,Newlyweds,"[Comedy, Romance, EdwardBurns, KerryBishé, Mar..."
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, date, loveat..."
4801,126186,Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton, DanielH..."


In [39]:
#Converting list into strings
df['tags'] = df['tags'].apply(lambda x:" ".join(x))

C:\Users\rajiv\AppData\Local\Temp\ipykernel_7324\4267108141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:" ".join(x))


In [40]:
df.head(3)

,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...


In [41]:
df['tags'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [42]:
#Converting into lower case
df['tags'] = df['tags'].apply(lambda x:x.lower())

C:\Users\rajiv\AppData\Local\Temp\ipykernel_7324\841579467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:x.lower())


In [43]:
df.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [44]:
import sklearn

In [45]:
#Converting text of tags into vectors using countvectorizer upto 5000 words and also using stop words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [46]:
vectors = cv.fit_transform(df['tags']).toarray()

In [47]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
#Name of most frequesnt 5000 words
cv.get_feature_names()

C:\Users\rajiv\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['16thcentury',
 '17thcentury',
 '18thcentury',
 '1910s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '19thcentury',
 '21stcentury',
 '3d',
 'aaroneckhart',
 'aarontaylor',
 'abbiecornish',
 'abhishekbachchan',
 'abigailbreslin',
 'aborigine',
 'abrams',
 'absurdism',
 'abuse',
 'abusivehusband',
 'accident',
 'action',
 'actionhero',
 'activist',
 'actor',
 'actress',
 'adambeach',
 'adambrody',
 'adamcarolla',
 'adamdriver',
 'adamgarcia',
 'adamgoldberg',
 'adammckay',
 'adamrifkin',
 'adamsandler',
 'adamscott',
 'adamshankman',
 'adaptation',
 'addiction',
 'adolescence',
 'adolfhitler',
 'adoptedchild',
 'adoption',
 'adoptivefather',
 'adrianlyne',
 'adrienbrody',
 'adriennebarbeau',
 'adultanimation',
 'adultery',
 'advancement',
 'adventure',
 'adversary',
 'advertisingexecutive',
 'advertisingexpert',
 'advice',
 'affair',
 'affection',
 'afghanistan',
 'africa',
 'africanamerican',
 'aftercreditsstinger',
 'afterlife',
 'agedifference',
 'agent',
 

In [50]:
!pip install nltk

In [51]:
import nltk

In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [53]:
#using stem function to convert similar words into one word (ex: activity and activities )
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [54]:
df['tags'] = df['tags'].apply(stem)

C:\Users\rajiv\AppData\Local\Temp\ipykernel_7324\866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [55]:
df['tags'][0]

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [56]:
#Using cosine distance function to find out the similarity btw two movie vectors by finding the distance btw them
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
similarity = cosine_similarity(vectors)

In [58]:
#sorting the similarity distance in reverse order upto 5 movies
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(47, 0.3849001794597506),
 (61, 0.3333333333333334),
 (83, 0.3142696805273545),
 (2372, 0.3142696805273545),
 (2403, 0.30123203803835463)]

In [59]:
#function to recommend movies 
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(df.iloc[i[0]].title)

In [60]:
recommend('Gandhi')

Winter in Wartime
Cry Freedom
A Bridge Too Far
Gandhi, My Father
Mandela: Long Walk to Freedom


In [61]:
import pickle

In [62]:
#using pickle to transfer movie list too the python environment 
pickle.dump(df.to_dict(), open('movie_dict.pkl','wb'))

In [63]:
df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [88]:
#using pickle to transfer similarity matrix too the python environment
pickle.dump(similarity,open('similarity.pkl','wb'))